In [1]:
from SymbolicDSGE import ModelConfig, ModelParser, DSGESolver
import sympy as sp
from warnings import catch_warnings, simplefilter
import numpy as np

In [2]:
conf = ModelParser("MODELS/test.yaml").get()

In [3]:
with catch_warnings():
    simplefilter(action="ignore")
    mat = sp.Matrix(conf.equations.model)
mat

Matrix([
[                Eq(Pi(t), beta*Pi(t + 1) + kappa*x(t))],
[Eq(x(t), -sigma*(-Pi(t + 1) + r(t)) + u(t) + x(t + 1))],
[                              Eq(u(t + 1), rho_u*u(t))],
[Eq(r_star(t), phi_pi*Pi(t) + phi_x*x(t) + rbar + v(t))],
[  Eq(r(t + 1), rho_r*r(t) + (1 - rho_r)*r_star(t + 1))],
[                              Eq(v(t + 1), rho_v*v(t))]])

In [4]:
sol = DSGESolver(conf)
comp = sol.compile(variable_order=conf.variables, n_state=3, n_exog=2)
conf.variables

[u, v, r, Pi, x, r_star]

In [5]:
solved = sol.solve(
    comp,
    steady_state=np.asarray([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=float),
    log_linear=False,
)

C:\Users\guney\Documents\GitHub\ThesisDraft\SymbolicDSGE\solver.py:245: ComplexWarning: Casting complex values to real discards the imaginary part
  p = np.asarray(mdl.p, dtype=float64)
C:\Users\guney\Documents\GitHub\ThesisDraft\SymbolicDSGE\solver.py:246: ComplexWarning: Casting complex values to real discards the imaginary part
  f = np.asarray(mdl.f, dtype=float64)


In [6]:
solved.policy.eig

array([5.29640671e-01+0.j, 6.99999918e-01+0.j, 4.99999963e-01+0.j,
       5.03611337e+16+0.j, 1.45329116e+00+0.j, 1.03671150e+00+0.j])

In [7]:
params = {
    p.name: float(conf.calibration.parameters[p])
    for p in conf.parameters
    if p in conf.calibration.parameters
}

# state at time t
s = np.array([0.05, 0.077, 0.06])  # or any test state
P = solved.policy.p
F = solved.policy.f

# controls at time t (jump variables)
c = F @ s

cur = np.concatenate([s, c])

# expected next state (NO shock)
s1 = P @ s
c1 = F @ s1
fwd = np.concatenate([s1, c1])

res = solved.compiled.equations(fwd, cur, params)
print(res)

[ 2.16704878e-09+0.j -2.26089439e-10+0.j -4.09781925e-09+0.j
 -1.97656912e-09+0.j -2.77087973e-09+0.j -2.86847348e-09+0.j]


In [15]:
solved.irf(["u", "v"], 10)

{'u': array([0.        , 1.        , 0.69999992, 0.48999989, 0.34299988,
        0.24009989, 0.1680699 , 0.11764892, 0.08235423, 0.05764796,
        0.04035356]),
 'v': array([0.        , 1.        , 0.49999996, 0.24999996, 0.12499997,
        0.06249998, 0.03124999, 0.01562499, 0.0078125 , 0.00390625,
        0.00195312]),
 'r': array([0.        , 0.        , 0.24720224, 0.29399905, 0.26487789,
        0.21421221, 0.16395461, 0.12156322, 0.08838157, 0.0634524 ,
        0.04517842]),
 'Pi': array([ 0.        ,  0.05780885,  0.01704091,  0.00051667, -0.00518507,
        -0.00631857, -0.00572284, -0.00463225, -0.00354317, -0.00262388,
        -0.00190503]),
 'x': array([ 0.        ,  1.70576747,  0.68872668,  0.2354124 ,  0.04459667,
        -0.02720672, -0.04737154, -0.04685458, -0.03939709, -0.03074587,
        -0.02303639]),
 'r_star': array([ 0.        ,  2.71065259,  1.17715366,  0.4700443 ,  0.15532697,
         0.0236128 , -0.02510965, -0.0379091 , -0.03644462, -0.03032876,
      